In [1]:
from DataReader import DataReaderChallangePolimi2017
import implicit
from CB_asy import CB_asymmetric_cosine
from CF_IB_asy import CF_IB_asymmetric_cosine
from CF_UB_asy import CF_UB_asymmetric_cosine
from implicit.nearest_neighbours import (BM25Recommender, CosineRecommender,TFIDFRecommender,ItemItemRecommender,all_pairs_knn)
from sklearn.preprocessing import normalize
import numpy as np
import scipy.sparse as sp
import pickle

evaluation = True
rebuild = False
dr = DataReaderChallangePolimi2017(evaluation=evaluation,rebuild=rebuild)

urm = dr.getURM_csr()
t_p = dr.target_playlists
t_t = dr.target_tracks

Date Reader Challange Polimi 2017
Loading data from ./DumpData/DataReaderTrainTestDump
Dataset train-test loaded


In [4]:
def bm25_row(self, X, K1=1.2, B=0.75):
    #Weighs each row of a sparse matrix by OkapiBM25 weighting
    # calculate idf per term (user)
    X = sp.coo_matrix(X)
    N = float(X.shape[0])
    idf = log(N / (1 + bincount(X.col)))

    # calculate length_norm per document (artist)
    row_sums = np.ravel(X.sum(axis=1))
    average_length = row_sums.mean()
    length_norm = (1.0 - B) + B * row_sums / average_length

    # weight matrix rows by bm25
    X.data = X.data * (K1 + 1.0) / (K1 * length_norm[X.row] + X.data) * idf[X.col]
    return X.tocsr()

In [5]:
#track-artist
icm = dr.getICM_csr(albums=False,artists=True,tags=False)
icm = normalize(icm,axis=0)
model = BM25Recommender(K=55)
model.fit(icm)
s = model.similarity
s.setdiag(0)
r_ar = urm*s.T
r_ar = normalize(r_ar, axis=0)
if evaluation: print dr.evaluateMAP(r_ar,verbose=False)
r_ar = dr.reduceRM(r_ar)

0.0702706944444


In [3]:
#track-artist
icm = dr.getICM_csr(albums=False,artists=True,tags=False)
icm = normalize(icm,axis=0)
icm = dr.bm25_row(icm)
s = all_pairs_knn(icm,55)
s.setdiag(0)
r_ar = urm*s.T
r_ar = normalize(r_ar, axis=0)
if evaluation: print dr.evaluateMAP(r_ar,verbose=False)
r_ar = dr.reduceRM(r_ar)

0.0702706944444


In [6]:
icm = dr.getICM_csr(albums=False,artists=True,tags=False).tocsr()

In [21]:
k=55
data, rows, cols = [], [], []
icm_t = icm.copy().T
print("Keep only k-similar item..")
for i_row in range(icm.shape[0]):
    if i_row%5000==0:
        print i_row
    row = icm[i_row, :]*icm_t
    idxs = np.array(row.indices)
    values = np.array(row.data)
    k_top = min(k, values.shape[0])
    topk_idxs_values = np.argpartition(values, -(k_top))[-(k_top):]
    n = topk_idxs_values.shape[0]
    # create incrementally the similarity matrix
    data.extend(values[topk_idxs_values])
    cols.extend(idxs[topk_idxs_values])
    rows.extend(np.full(n, i_row))
print("Building sparse matrix..")
s = sp.csr_matrix((data, (rows, cols)), shape=(icm.shape[0], icm.shape[0]), dtype=np.float32)

Keep only k-similar item..
0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
Building sparse matrix..


In [25]:
r_ar = urm*s.T
if evaluation: print dr.evaluateMAP(r_ar,verbose=False)

0.0430385555556


In [74]:
s2 = all_pairs_knn(icm,55)
s2.setdiag(0)
r_ar2 = urm*s2.T
r_ar2 = normalize(r_ar2, axis=0)
if evaluation: print dr.evaluateMAP(r_ar2,verbose=False)

0.0701556944444


In [75]:
print sum(s2.data)

2012454.0


In [62]:
def knn_fast1(m,k=100,verbose=True):
    data, rows, cols = [], [], []
    sim = (m * m.T).tocsr()
    for i_row in range(sim.shape[0]):
        if i_row%25000==0:
            print i_row
        row = sim[i_row, :]
        idxs = np.array(row.indices)
        values = np.array(row.data)
        k_top = min(k, values.shape[0])
        #topk_idxs_values = np.argpartition(values, -(k_top))[-(k_top):]
        topk_idxs_values = np.argsort(-values)[-(k_top):]
        n = topk_idxs_values.shape[0]
        # create incrementally the similarity matrix
        data.extend(values[topk_idxs_values])
        cols.extend(idxs[topk_idxs_values])
        rows.extend(np.full(n, i_row))
    sim = sp.csr_matrix((data, (rows, cols)), shape=(sim.shape[0], sim.shape[0]), dtype=np.float32)
    return sim

In [64]:
def knn_fast2(m,k=100,verbose=True):
    data, rows, cols = [], [], []
    m = m.tocsr()
    m_t = m.copy().T.tocsc()
    for i_row in range(m.shape[0]):
        if i_row%25000==0:
            print i_row
        row = m[i_row, :]*m_t
        idxs = np.array(row.indices)
        values = np.array(row.data)
        k_top = min(k, values.shape[0])
        #topk_idxs_values = np.argpartition(values, -(k_top))[-(k_top):]
        topk_idxs_values = np.argsort(-values)[-(k_top):]
        n = topk_idxs_values.shape[0]
        # create incrementally the similarity matrix
        data.extend(values[topk_idxs_values])
        cols.extend(idxs[topk_idxs_values])
        rows.extend(np.full(n, i_row))
    sim = sp.csr_matrix((data, (rows, cols)), shape=(m.shape[0], m.shape[0]), dtype=np.float32)
    return sim

In [69]:
def knn_slow1(m,k=100,verbose=True):
    data, rows, cols = [], [], []
    m = m.tocsr()
    sim = (m * m.T).tocsr()
    for i_row in range(m.shape[0]):
        if i_row%25000==0 and verbose:
            print i_row
        row = sim[i_row, :].toarray().ravel()
        topk_idxs_values = np.argpartition(row, -(k))[-(k):]
        #topk_idxs_values = np.argsort(row)[-(k):]
        n = topk_idxs_values.shape[0]
        # create incrementally the similarity matrix
        data.extend(row[topk_idxs_values])
        cols.extend(topk_idxs_values)
        rows.extend(np.full(n, i_row))
    s = sp.coo_matrix((data, (rows, cols)), shape=(m.shape[0], m.shape[0]), dtype=np.float32)
    s.eliminate_zeros()
    return s.tocsr()

In [40]:
def knn_slow2(m,k=100,verbose=True):
    data, rows, cols = [], [], []
    m = m.tocsr()
    m_t = m.copy().T.tocsc()
    for i_row in range(m.shape[0]):
        if i_row%10000==0 and verbose:
            print i_row
        row = (m[i_row, :]*m_t).toarray().ravel()
        topk_idxs_values = np.argpartition(row, -(k))[-(k):]
        n = topk_idxs_values.shape[0]
        # create incrementally the similarity matrix
        data.extend(row[topk_idxs_values])
        cols.extend(topk_idxs_values)
        rows.extend(np.full(n, i_row))
    s = sp.coo_matrix((data, (rows, cols)), shape=(m.shape[0], m.shape[0]), dtype=np.float32)
    s.eliminate_zeros()
    return s.tocsr()

In [70]:
s = knn_slow1(icm, k=55)
s.setdiag(0)
r_ar = urm*s.T
r_ar = normalize(r_ar, axis=0)
if evaluation: print dr.evaluateMAP(r_ar,verbose=False)

0
25000
50000
75000
0.07046025


In [71]:
print sum(s.data)

2012454.0


In [11]:
s = knn_slow2(icm, k=55)
s.setdiag(0)
r_ar = urm*s.T
r_ar = normalize(r_ar, axis=0)
if evaluation: print dr.evaluateMAP(r_ar,verbose=False)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
0.07046025


In [72]:
s = knn_fast1(icm, k=55)
s.setdiag(0)
r_ar = urm*s.T
r_ar = normalize(r_ar, axis=0)
if evaluation: print dr.evaluateMAP(r_ar,verbose=False)

Keep only k-similar item..
0
25000
50000
75000
Building sparse matrix..
0.0700929166667


In [73]:
print sum(s.data)

2012454.0


In [65]:
s = knn_fast2(icm, k=55)
s.setdiag(0)
r_ar = urm*s.T
r_ar = normalize(r_ar, axis=0)
if evaluation: print dr.evaluateMAP(r_ar,verbose=False)

0
25000
50000
75000
0.0700929166667


In [199]:
k=55
data, rows, cols = [], [], []
icm = sp.csr_matrix(icm,dtype=np.float)
sim = (icm * icm.T).tocsr()
print("Keep only k-similar item..")
for i_row in range(icm.shape[0]):
    if i_row%25000==0:
        print i_row
    row = sim[i_row, :]
    idxs = np.array(row.indices)
    values = np.array(row.data)
    k_top = min(k, values.shape[0])
    topk_idxs_values = np.argpartition(values, -(k_top))[-(k_top):]
    n = topk_idxs_values.shape[0]
    # create incrementally the similarity matrix
    data.extend(values[topk_idxs_values])
    cols.extend(idxs[topk_idxs_values])
    rows.extend(np.full(n, i_row))
print("Building sparse matrix..")
s = sp.csr_matrix((data, (rows, cols)), shape=(icm.shape[0], icm.shape[0]), dtype=np.float32)

Keep only k-similar item..
0
25000
50000
75000
Building sparse matrix..


In [200]:
s.setdiag(0)
r_ar = urm*s.T
r_ar = normalize(r_ar, axis=0)
if evaluation: print dr.evaluateMAP(r_ar,verbose=False)

0.0699937777778
